In [1]:
import numpy as np
import pandas as pd
import os
import sys as sys
import socket
import matplotlib.pyplot as plt
import multiprocessing as mp
import itertools as it
#from adjustText import adjust_text

import warnings
warnings.filterwarnings("ignore") # Great Style
socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("No compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns

print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

#sys.path.insert(0,"/n/groups/reich/hringbauer/git/hapBLOCK/package/")  # hack to get development package first in path
from ancIBD.ibd_stats.funcs import new_columns, find_relatives, give_sub_df, plot_age_diff, rc_date

compute-a-16-168.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 32


# 1) Browse full IBD dataset (Eurasia)

In [18]:
#path_save = f"/n/groups/reich/hringbauer/git/punic_aDNA/output/ibd/v54.1.ibd_ind.d220.tsv" # for IBD csv file
path_save = f"/n/groups/reich/hringbauer/git/ibd_euro/output/ibd/ibd220.ind.v56.3.tsv" # for IBD csv file
df = pd.read_csv(path_save, sep="\t")

#path_save = "/n/groups/reich/hringbauer/Data/v54.1.anno.csv"# for IBD csv file
path_save = "/n/groups/reich/hringbauer/Data/v56.3.anno.csv"# for IBD csv file
df_meta = pd.read_csv(path_save, sep=",")

new_columns(df, df_meta, col = ["age", "loc", "n_cov_snp","clst"]) #"clst"

df=df.sort_values(by="sum_IBD>20", ascending=False)
df20 = df[df["max_IBD"]>16].copy()

In [16]:
give_sub_df(df, pop1='Kerkouane', pop2='', col='loc').sort_values(by="max_IBD", ascending=False)[:50]

Found: 718 Pairs fitting in dataframe.



,iid1,iid2,max_IBD,sum_IBD>8,n_IBD>8,sum_IBD>12,n_IBD>12,sum_IBD>16,n_IBD>16,sum_IBD>20,n_IBD>20,age1,age2,loc1,loc2,n_cov_snp1,n_cov_snp2
267,R11780.SG,I24494,283.652203,3333.716194,20.0,3333.716194,20.0,3333.716194,20.0,3333.716194,20.0,2547.0,2464.0,Kerkouane,Kerkouane,860682,791516
989,I24494,I24193,283.652203,2005.149298,32.0,1985.909503,30.0,1958.025506,28.0,1922.099313,26.0,2464.0,2500.0,Kerkouane,Kerkouane,791516,789887
2047,R11780.SG,I24193,128.366596,1003.366010,32.0,946.618403,26.0,920.824103,24.0,830.916093,19.0,2547.0,2500.0,Kerkouane,Kerkouane,860682,789887
3298,I24494,I24215,92.417502,477.516500,10.0,469.002405,9.0,469.002405,9.0,469.002405,9.0,2464.0,2505.0,Kerkouane,Kerkouane,791516,1085353
4274,I24193,I24215,90.698195,316.495398,14.0,284.886196,11.0,230.299905,7.0,194.081602,5.0,2500.0,2505.0,Kerkouane,Kerkouane,789887,1085353
2561,I24194,I24193,77.237201,732.451471,22.0,713.814671,20.0,673.944468,17.0,657.024176,16.0,2500.0,2500.0,Kerkouane,Kerkouane,791265,789887
2611,I24194,I24494,75.874001,714.631484,16.0,696.773581,14.0,696.773581,14.0,679.805879,13.0,2500.0,2464.0,Kerkouane,Kerkouane,791265,791516
4445,R11780.SG,I24194,74.766899,296.729207,10.0,256.621492,6.0,256.621492,6.0,256.621492,6.0,2547.0,2500.0,Kerkouane,Kerkouane,860682,791265
5784,R11780.SG,I24215,62.770103,205.133484,6.0,205.133484,6.0,205.133484,6.0,186.140393,5.0,2547.0,2505.0,Kerkouane,Kerkouane,860682,1085353
9205,I24194,I24215,54.115105,120.898995,6.0,110.503897,5.0,54.115105,1.0,54.115105,1.0,2500.0,2505.0,Kerkouane,Kerkouane,791265,1085353


In [20]:
give_sub_df(df, pop1='I24494', pop2='', col='iid').sort_values(by="max_IBD", ascending=False)[:50]

Found: 35 Pairs fitting in dataframe.



,iid1,iid2,max_IBD,sum_IBD>8,n_IBD>8,sum_IBD>12,n_IBD>12,sum_IBD>16,n_IBD>16,sum_IBD>20,n_IBD>20,age1,age2,loc1,loc2,n_cov_snp1,n_cov_snp2,clst1,clst2
267,R11780.SG,I24494,283.652203,3333.716194,20.0,3333.716194,20.0,3333.716194,20.0,3333.716194,20.0,2547.0,2464.0,Kerkouane,Kerkouane,860682,791516,Tunisia_Punic.SG,Tunisia_Punic_mother.or.daughter.I24040
989,I24494,I24193,283.652203,2005.149298,32.0,1985.909503,30.0,1958.025506,28.0,1922.099313,26.0,2464.0,2500.0,Kerkouane,Kerkouane,791516,789887,Tunisia_Punic_mother.or.daughter.I24040,Tunisia_Punic
3298,I24494,I24215,92.417502,477.516500,10.0,469.002405,9.0,469.002405,9.0,469.002405,9.0,2464.0,2505.0,Kerkouane,Kerkouane,791516,1085353,Tunisia_Punic_mother.or.daughter.I24040,Tunisia_Punic_oAfrica
2611,I24194,I24494,75.874001,714.631484,16.0,696.773581,14.0,696.773581,14.0,679.805879,13.0,2500.0,2464.0,Kerkouane,Kerkouane,791265,791516,Tunisia_Punic,Tunisia_Punic_mother.or.daughter.I24040
157994,I13834,I24494,16.408801,16.408801,1.0,16.408801,1.0,16.408801,1.0,0.000000,0.0,531.0,2464.0,"Barç (Southeast, Korça Basin)",Kerkouane,879052,791516,Albania_EarlyModern_oEuropean,Tunisia_Punic_mother.or.daughter.I24040
169183,I25640,I24494,14.917707,14.917707,1.0,14.917707,1.0,0.000000,0.0,0.000000,0.0,2488.0,2464.0,"Campania, San Marzano",Kerkouane,600243,791516,Italy_Campania_IA_B,Tunisia_Punic_mother.or.daughter.I24040
199779,I13364,I24494,12.707496,12.707496,1.0,12.707496,1.0,0.000000,0.0,0.000000,0.0,2400.0,2464.0,"Emilia Romagna, Montericco, Imola",Kerkouane,797923,791516,Italy_North_IA_Umbri,Tunisia_Punic_mother.or.daughter.I24040
200322,I24494,I31676,12.679207,12.679207,1.0,12.679207,1.0,0.000000,0.0,0.000000,0.0,2464.0,2650.0,Kerkouane,"Lazio, Rieti, Corvaro",791516,1096130,Tunisia_Punic_mother.or.daughter.I24040,Italy_EIA_Equi
99687,I34701,I24494,12.179196,21.265197,2.0,12.179196,1.0,0.000000,0.0,0.000000,0.0,2850.0,2464.0,"Abruzzo, L'Aquila, Pizzoli",Kerkouane,1010733,791516,Italy_IA,Tunisia_Punic_mother.or.daughter.I24040
217736,R11746.SG,I24494,11.932594,11.932594,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2485.0,2464.0,Kerkouane,Kerkouane,792358,791516,Tunisia_Punic_oAfrica1.SG,Tunisia_Punic_mother.or.daughter.I24040


# 2) Browse Punic-specific IBD Dataset
Contains more low coverage Indidivuals

In [4]:
df = pd.read_csv("/n/groups/reich/hringbauer/git/punic_aDNA/output/ibd/v54.1.ibd_ind.d220.tsv", sep="\t")
path_save = "/n/groups/reich/hringbauer/Data/v54.1.anno.csv"# for IBD csv file
df_meta = pd.read_csv(path_save, sep=",")

new_columns(df, df_meta, col = ["age", "loc", "n_cov_snp"]) #"clst"

,iid1,iid2,max_IBD,sum_IBD>8,n_IBD>8,sum_IBD>12,n_IBD>12,sum_IBD>16,n_IBD>16,sum_IBD>20,n_IBD>20,age1,age2,loc1,loc2,n_cov_snp1,n_cov_snp2
0,I35351,I35333,282.987110,3332.207697,21.0,3332.207697,21.0,3332.207697,21.0,3332.207697,21.0,2500.0,2500.0,"Cap Bon, Kelibia","Cap Bon, Kelibia",1028493,986415
1,I24494,I24040,283.652203,3326.638294,20.0,3326.638294,20.0,3326.638294,20.0,3326.638294,20.0,2464.0,2547.0,Kerkouene,Kerkouene,791516,702598
2,I22122,I22118,213.983188,3308.844301,25.0,3308.844301,25.0,3308.844301,25.0,3274.698507,23.0,2419.0,2589.0,"Sardinia, Tharros","Sardinia, Tharros",711166,802359
3,I24494,I24193,283.652203,2048.808393,33.0,2029.570302,31.0,2001.710003,29.0,1966.079616,27.0,2464.0,2500.0,Kerkouene,Kerkouene,791516,789887
4,I22122,I22117,176.386502,1706.853619,24.0,1706.853619,24.0,1706.853619,24.0,1706.853619,24.0,2419.0,2582.0,"Sardinia, Tharros","Sardinia, Tharros",711166,791179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,I22252,I28504,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2600.0,2337.0,"Akhziv, southern cemetery",Carthage,530139,679772
359,I22252,I22260,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2600.0,2600.0,"Akhziv, southern cemetery","Akhziv, southern cemetery",530139,600271
360,I22236,I35334,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2528.0,2500.0,"Sicily, Motya","Cap Bon, Kelibia",799650,1080851
361,I22236,I24045,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2528.0,2500.0,"Sicily, Motya",Kerkouene,799650,696534


In [5]:
iids = ["18199", "I18201", "I18203"]


In [6]:
df[df["iid1"].isin(iids) | df["iid2"].isin(iids)]

,iid1,iid2,max_IBD,sum_IBD>8,n_IBD>8,sum_IBD>12,n_IBD>12,sum_IBD>16,n_IBD>16,sum_IBD>20,n_IBD>20,age1,age2,loc1,loc2,n_cov_snp1,n_cov_snp2
6,I18199,I18203,87.207204,762.694800,36.0,646.692397,24.0,518.093885,15.0,465.146892,12.0,2226.0,2225.0,"Almería, Cuevas del Almazora, Villaricos","Almería, Cuevas del Almazora, Villaricos",744521,591457
9,I18199,I18201,47.389899,650.968208,36.0,515.483997,22.0,417.833296,15.0,350.126997,11.0,2226.0,2554.0,"Almería, Cuevas del Almazora, Villaricos","Almería, Cuevas del Almazora, Villaricos",744521,881415
12,I18201,I18203,27.386105,313.902313,24.0,174.109095,10.0,89.933786,4.0,72.560105,3.0,2554.0,2225.0,"Almería, Cuevas del Almazora, Villaricos","Almería, Cuevas del Almazora, Villaricos",881415,591457
27,I26932,I18203,20.142701,36.934009,2.0,36.934009,2.0,36.934009,2.0,20.142701,1.0,2230.0,2225.0,"Málaga, Calle Mármoles","Almería, Cuevas del Almazora, Villaricos",822991,591457
36,I26932,I18201,10.000396,28.411689,3.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2230.0,2554.0,"Málaga, Calle Mármoles","Almería, Cuevas del Almazora, Villaricos",822991,881415
81,I26931,I18203,11.211300,11.211300,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2239.0,2225.0,"Málaga, Calle Mármoles","Almería, Cuevas del Almazora, Villaricos",747809,591457
82,I22118,I18201,11.209595,11.209595,1.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2589.0,2554.0,"Sardinia, Tharros","Almería, Cuevas del Almazora, Villaricos",802359,881415
195,I26842,I18201,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2350.0,2554.0,"Málaga, Hipgeo del Parking de Alcazabilla","Almería, Cuevas del Almazora, Villaricos",824318,881415
200,I26915,I18201,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2350.0,2554.0,"Málaga, Hipgeo del Parking de Alcazabilla","Almería, Cuevas del Almazora, Villaricos",743832,881415
201,I26915,I18203,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2350.0,2225.0,"Málaga, Hipgeo del Parking de Alcazabilla","Almería, Cuevas del Almazora, Villaricos",743832,591457


# Browse All IBD